In [63]:
sys.path.append(r"C:\Users\Minji Kang\Documents\GitHub\network_reliability")

import networkx as nx
import matplotlib.pyplot as plt
from BNS_JT import cpm, variable, operation
from networkx.algorithms.flow import shortest_augmenting_path
from BNS_JT import brc
import numpy as np
import math
import random


### **Network topology**

In [64]:
nodes = {
    "n1": (0, 0), "n2": (1.5, 5), "n3": (2.8, 2.5), "n4": (2.5, 0),
    "n5": (4.6, 7), "n6": (5.5, -0.5), "n7": (6.5, 3), "n8": (7, -1)
}

edges = {
    "e1": ("n1", "n2"), "e2": ("n1", "n3"), "e3": ("n1", "n4"),
    "e4": ("n2", "n3"), "e5": ("n3", "n4"), "e6": ("n5", "n2"),
    "e7": ("n5", "n3"), "e8": ("n3", "n7"), "e9": ("n3", "n6"), 
    "e10": ("n6", "n4"), "e11": ("n5", "n7"), "e12": ("n7", "n6"), 
    "e13": ("n6", "n8"), "e14": ("n7", "n8")
}

G = nx.Graph() 
for node, position in nodes.items():
    G.add_node(node, pos=position)
for edge_id, (u, v) in edges.items():
    pos_u, pos_v = nodes[u], nodes[v]
    distance = math.sqrt((pos_u[0] - pos_v[0])**2 + (pos_u[1] - pos_v[1])**2)
    G.add_edge(u, v, label=edge_id, flow=0, capacity=2, weight=distance)  # 노드와 노드 사이 거리를 가중치로 설정하여 최단경로탐색 시 더 짧은 거리를 우선순위로 함
                                                                          # Issue 1: edge 별 capacity 설정 기준이 없음. 임의의 capacity 2 설정
        
plt.figure(figsize=(5, 4))
pos = nx.get_node_attributes(G, 'pos')
edge_labels = nx.get_edge_attributes(G, 'label') 
nx.draw(G, pos, with_labels=True, node_size=800, node_color="lightblue", font_size=10, font_weight="bold")
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, label_pos=0.5)
plt.title("Railway Network", fontsize=15)
plt.show()

### **Component events**

In [65]:
varis = {}
for k, v in edges.items():
    varis[k] = variable.Variable( name=k, values = [0, 1]) # values: edge flow capacity

print(varis['e1'])

probs = {'e1': {0: 0.01, 1: 0.99}, 'e2': {0: 0.01, 1: 0.99}, 'e3': {0: 0.05, 1: 0.95},
         'e4': {0: 0.05, 1: 0.95}, 'e5': {0: 0.10, 1: 0.90}, 'e6': {0: 0.01, 1: 0.99},
         'e7': {0: 0.01, 1: 0.99}, 'e8': {0: 0.01, 1: 0.99}, 'e9': {0: 0.05, 1: 0.95},
         'e10': {0: 0.01, 1: 0.99}, 'e11': {0: 0.01, 1: 0.99}, 'e12': {0: 0.05, 1: 0.95},
         'e13': {0: 0.01, 1: 0.99}, 'e14': {0: 0.01, 1: 0.99}}

'Variable(name=e1, B=[{0}, {1}, {0, 1}], values=[0, 1])'


### **System event**

Analyze multi-OD pairs connectivity and flow in a network with edge failures.

Input:
- comps_st: dict, component states (e.g., edge operational states)
- edges: dict, edge definitions (key=edge id, value=(start node, end node))
- varis: dict, variable definitions for edge capacities
- od_pairs: list, list of (source, target) OD pairs
- n_additional_failures: int, number of additional edges to fail

Returns:
- final_failed_edges: list, final list of failed edge IDs
- edge_flows: dict, edge flow and capacity status after processing OD pairs


**Issue**
- return 할 때 f_val, sys_st, min_comps_st 반환 필요. od pair 1개일 때는 source부터 terminal까지의 path가 없으면 system fail. 그렇다면 od pair가 여러개일 때는 system fail을 어떻게 정의할 것인가?**


In [ ]:
def multicommodity_net_conn(comps_st, edges, varis, od_pairs, n_additional_failures=3):

    # Initialize network graph
    G = nx.Graph()
    for k, x in comps_st.items():
        G.add_edge(edges[k][0], edges[k][1])
        G[edges[k][0]][edges[k][1]]['capacity'] = varis[k].values[x]
        G[edges[k][0]][edges[k][1]]['flow'] = 0  # Initialize flow to 0
        G[edges[k][0]][edges[k][1]]['label'] = k  # Add edge ID label for reference

    # Process OD pairs and update flow
    failed_edges = set()
    for s, t in od_pairs:
        if nx.has_path(G, s, t):
            path = nx.shortest_path(G, source=s, target=t, weight='weight')
            for i in range(len(path) - 1):
                u, v = path[i], path[i + 1]
                G[u][v]['flow'] += 1
                if G[u][v]['flow'] > G[u][v]['capacity']:
                    failed_edges.add(G[u][v]['label'])
        else:
            print(f"OD Pair ({s}, {t}) is NOT connected.")
    
    ## (for checking results)
    print("\nEdge States After Processing All OD Pairs:")
    for u, v, data in G.edges(data=True):
        if data['flow'] > data['capacity']:
            print(f"Edge ({u}, {v}): Flow={data['flow']}, Capacity={data['capacity']} - exceeded capacity!")
            failed_edges.add(data['label']) 
        else:
            print(f"Edge ({u}, {v}): Flow={data['flow']}, Capacity={data['capacity']}")

    # Collect failed edges after initial OD pair processing
    failed_edges_list = list(failed_edges)
    print(f"\nInitial Failed Edges: {failed_edges_list}")

    # Identify additional edges to fail
    all_edge_ids = [data['label'] for _, _, data in G.edges(data=True)]
    available_edge_ids = [edge_id for edge_id in all_edge_ids if edge_id not in failed_edges]
    additional_failed_edges = random.sample(available_edge_ids, n_additional_failures)
    print(f"Additional Failed Edges: {additional_failed_edges}")

    # Remove failed edges from the graph
    for edge_id in failed_edges_list + additional_failed_edges:
        failed_edge = None
        for u, v, data in G.edges(data=True):
            if data['label'] == edge_id:
                failed_edge = (u, v)
                break
        if failed_edge and G.has_edge(*failed_edge):
            G.remove_edge(*failed_edge)

    # Output final failed edges and edge flows
    final_failed_edges = failed_edges_list + additional_failed_edges
    edge_flows = {f"{u}-{v}": {'flow': data['flow'], 'capacity': data['capacity']}
                  for u, v, data in G.edges(data=True)}
    print
    return final_failed_edges, edge_flows

In [67]:
sys_fun = lambda comps_st : multicommodity_net_conn(comps_st, edges, varis, od_pairs, n_additional_failures=3)

### **Generate random od pairs**

In [68]:
od_pairs = []
node_list = list(G.nodes)

while len(OD_pairs) < 5:  # OD 쌍 개수 설정
                          # Issue 2: 수요에 대한 값을 어떻게 정할 것인가?
    s = random.choice(node_list)  
    t = random.choice(node_list) 
    if s != t:  
        OD_pairs.append((s, t))

### **Run BRC**

In [69]:
brs, rules, sys_res, monitor = brc.run(varis, probs, sys_fun, max_sf=np.inf, max_nb=np.inf, pf_bnd_wr=0.0)


Edge States After Processing All OD Pairs:
Edge (n1, n2): Flow=0, Capacity=1
Edge (n1, n3): Flow=0, Capacity=1
Edge (n1, n4): Flow=0, Capacity=1
Edge (n2, n3): Flow=0, Capacity=1
Edge (n2, n5): Flow=0, Capacity=1
Edge (n3, n4): Flow=0, Capacity=1
Edge (n3, n5): Flow=0, Capacity=1
Edge (n3, n7): Flow=0, Capacity=1
Edge (n3, n6): Flow=0, Capacity=1
Edge (n4, n6): Flow=0, Capacity=1
Edge (n5, n7): Flow=0, Capacity=1
Edge (n7, n6): Flow=0, Capacity=1
Edge (n7, n8): Flow=0, Capacity=1
Edge (n6, n8): Flow=0, Capacity=1

Initial Failed Edges: []
Additional Failed Edges: ['e2', 'e10', 'e5']


ValueError: not enough values to unpack (expected 3, got 2)